# AI Assisted Annotation 

Manual annotation is slow, tedious and costly. Faster labeling of 3D volumes using AI annotation models accelerates this process. Clara Train offers an AIAA API that easily integrates into common medical imaging viewers
<br><img src="screenShots/AIAASpeedup.png" alt="Drawing" style="height: 400px;"/><br>

AIAA is based on a server client model as shown below 
<br><img src="screenShots/AIAAClientServer.png" alt="Drawing" style="height: 400px;"/><br>



By the end of this notebook you will be able to:
- Start AIAA server 
- Load a deep grow model
- Annotate using deep grow 
- Load your model and use it for annotations  
- Stop AIAA server


## Prerequisites
- Nvidia GPU with 8Gb of memory   


### Resources
It might be helpful to watch the GTC Digital 2021 talk on Clara Train SDK
- [Clara Train 4.0 - 101 Getting Started [SE2688]](https://gtc21.event.nvidia.com/media/Clara%20Train%204.0%20-%20101%20Getting%20Started%20%5BSE2688%5D/1_0qgfrql2) 
Clara train Getting started: cover basics, BYOC, AIAA, AutoML 


# Lets get started

To interact with AIAA from the command line you should use the AIAA as `AIAA -h`. 
This would show you all commands as shown below
```
Usage: AIAA [-h] {start,stop,status,list,load,delete} ...

commands:
  start   Start AIAA Server
          AIAA start -h
  stop    Stop AIAA Server
  status  Check if AIAA Server is running
  list    List all models loaded in AIAA
  logs    Fetch AIAA Logs if server is running <lines>
  load    Load a model to AIAA using one of the sources {ngc|mmar|zip|config}
          sources:
            ngc      Load model from NGC
                     AIAA load <model> ngc <path> <version>
            mmar     Load model from MMAR Folder
                     AIAA load <model> mmar <path>
            zip      Load model from ZIP Archive
                     AIAA load <model> zip <path>
            config   Load model using AIAA Config and Model saved file
                     AIAA load <model> config <aiaa_config> <saved_model>
            pipeline Load pipeline (as vitual model) using AIAA Config
                     AIAA load <model> config <aiaa_config>
  delete  Delete a model from AIAA
          AIAA delete <model>
```

Before we get started let us check that we have an NVIDIA GPU available in the docker by running the cell below

In [1]:
# following command should show all gpus available 
!nvidia-smi

Fri Aug  6 11:01:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 106...  On   | 00000000:01:00.0  On |                  N/A |
| N/A   68C    P2    25W /  N/A |   1163MiB /  6078MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------


# 1. Start AIAA server
First lets set up AIAA path and change some permissions 

In [4]:
AIAA_ROOT="/claraDevDay/AIAA/workspace/"
AIAA_PORT="5000"
!mkdir -p $AIAA_ROOT
!chmod 777 $AIAA_ROOT
print ("AIAA_ROOT is set to ",AIAA_ROOT)

AIAA_ROOT is set to  /claraDevDay/AIAA/workspace/


##  1.2 AIAA Server with Triton BackEnd 
Starting V4 Triton server was moved out of the clara train container. 
In order to use Triton as the AIAA backend, you should use `restartClaraTrain.sh`. 
The script uses docker-compose to start both the clara train container and triton container, 
while connected them through the docker internal network. 

Now lets start the AIAA server by running the cell below

In [13]:
! AIAA start -b -w $AIAA_ROOT --engine TRITON --triton_ip tritonserver 
#! AIAA start -b -w $AIAA_ROOT --engine TRITON --triton_ip 0.0.0.0 --triton_http_port 3031

AIAA:: Current User: root
AIAA:: RUNNING AIAA AS: nvidia:nvidia

AIAA:: RUN FILE: /claraDevDay/AIAA/workspace/aiaa-run-config.json
AIAA:: 
AIAA:: ENGINE:: engine=TRITON
AIAA:: *****************************************
AIAA:: Using TRITON Backend
AIAA:: 
AIAA:: TRITON:: triton_ip=tritonserver
AIAA:: TRITON:: triton_http_port=8000
AIAA:: TRITON:: triton_grpc_port=8001
AIAA:: TRITON:: triton_metrics_port=8002
AIAA:: TRITON:: triton_proto=http
AIAA:: TRITON:: triton_shmem=no
AIAA:: TRITON:: triton_model_path=/claraDevDay/AIAA/workspace//triton_models
AIAA:: TRITON:: triton_verbose=0
AIAA:: TRITON:: triton_start_timeout=120
AIAA:: TRITON:: triton_model_timeout=30
AIAA:: 
AIAA:: Checking Triton Server is running on: tritonserver:8000
AIAA:: Triton is up and running OK
AIAA:: *****************************************
AIAA:: 
AIAA:: APACHE:: LOG  DIR=/claraDevDay/AIAA/workspace//logs/0
AIAA:: APACHE:: RUN  DIR=/claraDevDay/AIAA/workspace//apache/0/run
AIAA:: APACHE:: LOCK DIR=/claraDevDay/AIAA

In [14]:
!cat /etc/apache2/ports.conf

# If you just change the port or add more ports here, you will likely also
# have to change the VirtualHost statement in
# /etc/apache2/sites-enabled/000-default.conf

Listen 5000

<IfModule ssl_module>
	Listen 5001
</IfModule>

<IfModule mod_gnutls.c>
	Listen 5001
</IfModule>

# vim: syntax=apache ts=4 sw=4 sts=4 sr noet


In [12]:
# Stop triton server
# !AIAA stop

AIAA:: AIAA Server stopped!!!


# 2 Check on AIAA Server 

## 2.1 using CLI
You should check on the AIAA status

In [5]:
!AIAA status   

AIAA Server is Up and Running [Backend Engine: TRITON]; Total Loaded Models: 3
TRITON Server is Up and Running


You can also get the last 15 lines of the logs use

In [5]:
!AIAA logs 15 

[2021-08-04 14:19:19] [INFO] (aiaa.utils.generic_utils) - Reading Model Configs from /claraDevDay/AIAA/workspace/aiaa-models-config.json
[2021-08-04 14:19:19] [INFO] (aiaa.www.api.server_context) - Total Models Loaded = 0
[2021-08-04 14:19:23] [INFO] (aiaa.utils.generic_utils) - Reading Model Configs from /claraDevDay/AIAA/workspace/aiaa-models-config.json
[2021-08-04 14:19:23] [INFO] (aiaa.www.api.server_context) - Total Models Loaded = 0
[2021-08-04 14:51:18] [INFO] (aiaa.utils.generic_utils) - Reading Model Configs from /claraDevDay/AIAA/workspace/aiaa-models-config.json
[2021-08-04 14:51:18] [INFO] (aiaa.www.api.server_context) - Total Models Loaded = 0
[2021-08-04 14:51:22] [INFO] (aiaa.utils.generic_utils) - Reading Model Configs from /claraDevDay/AIAA/workspace/aiaa-models-config.json
[2021-08-04 14:51:22] [INFO] (aiaa.www.api.server_context) - Total Models Loaded = 0


You can then easily filter on errors as  

In [6]:
!AIAA logs |grep errors

To list models you can run 

In [7]:
!AIAA list | grep name

In [6]:
!AIAA list

[]



### 2.2 Using Browser
You can also check on the AIAA server using your browser by checking:
- Main url `http://localhost:5000/` <br>
- APIS to list, upload, delete models `http://localhost:5000/docs/`
- Check logs `http://localhost:5000/logs`
- List available models `http://localhost:5000/v1/models/` 
 

### 2.3 (For advanced users) Using curl commands
You can also use curl command to do list logs 
The cell below will get the last 15 lines of the logs. 

In [15]:
http_str="http://127.0.0.1:"+AIAA_PORT+"/logs/?lines=15"
!curl -X GET $http_str -H "accept: application/json" 

[2021-08-06 14:14:44] [INFO] (aiaa.inference.inference_utils) - PRE - Input Keys: dict_keys(['result_extension', 'result_dtype', 'result_compress', 'image_original', 'image', 'image_path', 'params', 'transform_meta'])
[2021-08-06 14:14:48] [INFO] (aiaa.inference.inference_utils) - PRE - Transform (LoadImaged): Time: 3.1971; image: (512, 512, 111)
[2021-08-06 14:14:48] [INFO] (aiaa.inference.inference_utils) - PRE - Transform (AddChanneld): Time: 0.0000; image: (1, 512, 512, 111)
[2021-08-06 14:15:00] [INFO] (aiaa.inference.inference_utils) - PRE - Transform (Spacingd): Time: 12.7835; image: (1, 500, 500, 331)
[2021-08-06 14:15:26] [INFO] (aiaa.inference.inference_utils) - PRE - Transform (ScaleIntensityRanged): Time: 25.9662; image: (1, 500, 500, 331)
[2021-08-06 14:15:26] [INFO] (aiaa.inference.triton_inference) - Input  Shape: (1, 1, 500, 500, 331), Type: ndarray
[2021-08-06 14:15:35] Truncated or oversized response headers received from daemon process 'AIAA_V1': /opt/nvidia/medical/

To list models

In [16]:
http_str="http://127.0.0.1:"+AIAA_PORT+"/v1/models"
# http://18.132.85.178:5000/:"+AIAA_PORT+"/v1/models"
!curl -X GET $http_str -H "accept: application/json"

[{"name": "deepgrow3d", "labels": [], "description": "3D DeepGrow model based on Unet", "version": "1", "type": "deepgrow", "deepgrow": "3D"}, {"name": "spleen_seg", "labels": ["spleen"], "description": "A pre-trained model for volumetric (3D) segmentation of the spleen from CT image", "version": "1", "type": "segmentation"}]

#  3. Load models in AIAA Server. 
For this notebook we can download models from [NGC](). Models on NGC are either:
- Annotation Models <br>
[Deep Extreme Cut: From Extreme Points to Object Segmentation](https://arxiv.org/abs/1711.09081)
<br><img src="screenShots/AIAAAnnotation.png" alt="Drawing" style="height: 400px;"/><br>

- Segmentation Models
- DeepGrow MMAR 
This is an interactive model to get you started with annotation. CNN takes in single channel (image) + use single click 
for foreground or background location then produces the segmentation. it is based on [Interactive segmentation of medical images through
fully convolution neural networks](https://arxiv.org/pdf/1903.08205.pdf)
<br><img src="screenShots/AIAADeepGrow.png" alt="Drawing" style="height: 400px;"/><br>


## 3.0 Loading options
In order to load a model to AIAA we will use the AIAA cli as 

In [11]:
! AIAA load -h

Loading Model: -h
Usage: AIAA load <model> {ngc|mmar|zip|config|pipeline} ...


This will show you options which allows to load from ngc or a local MMAR or a local zip file, etc 
```
root@claratrain:/claraDevDay/AIAA# AIAA load -h
Loading Model: -h
Usage: AIAA load <model> {ngc|mmar|zip|config|pipeline} ...
```

## 3.1 Load model trained from Getting started notebook
In order to load model trained in the getting started notebook you can run

In [17]:
modelName="Test"
modelFolderPath="/claraDevDay/GettingStarted/models/config_train_Unet/model.ts"
configFolderPath="/claraDevDay/GettingStarted/config/config_aiaa.json"
!AIAA load $modelName config $configFolderPath $modelFolderPath 


Loading Model: Test
Model Saved File: /claraDevDay/GettingStarted/models/config_train_Unet/model.ts does not exist.


## 3.2 Using NGC CLI to download model from NGC
You can see a [list of available pre-trained models](https://ngc.nvidia.com/containers/nvidia:clara-train-sdk) on NGC. 
You can also use `ngc registry model list nvidia/med/clara_*` to get a list of models.

In [6]:
!ngc registry model list nvidia/med/clara_pt*

+---------+---------+---------+---------+---------+---------+---------+---------+
| Name    | Reposit | Latest  | Applica | Framewo | Precisi | Last Mo | Permiss |
|         | ory     | Version | tion    | rk      | on      | dified  | ion     |
+---------+---------+---------+---------+---------+---------+---------+---------+
| clara_p | nvidia/ | 1       | Segment | Clara   | AMP     | May 20, | unlocke |
| t_brain | med/cla |         | ation   | Train   |         | 2021    | d       |
| _mri_se | ra_pt_b |         |         |         |         |         |         |
| gmentat | rain_mr |         |         |         |         |         |         |
| ion     | i_segme |         |         |         |         |         |         |
|         | ntation |         |         |         |         |         |         |
| clara_p | nvidia/ | 1       | Segment | Clara   | AMP     | May 20, | unlocke |
| t_brain | med/cla |         | ation   | Train   |         | 2021    | d       |
| _mri_se | ra_p

In [8]:
# grab lung
!ngc registry model list nvidia/med/*lung*

+---------+---------+---------+---------+---------+---------+---------+---------+
| Name    | Reposit | Latest  | Applica | Framewo | Precisi | Last Mo | Permiss |
|         | ory     | Version | tion    | rk      | on      | dified  | ion     |
+---------+---------+---------+---------+---------+---------+---------+---------+
| clara_t | nvidia/ | 1       | Annotat | Medical | FP32    | Mar 03, | unlocke |
| rain_co | med/cla |         | ion     |         |         | 2021    | d       |
| vid19_a | ra_trai |         |         |         |         |         |         |
| nnotati | n_covid |         |         |         |         |         |         |
| on_ct_l | 19_anno |         |         |         |         |         |         |
| ung     | tation_ |         |         |         |         |         |         |
|         | ct_lung |         |         |         |         |         |         |
| clara_t | nvidia/ | 1       | Segment | Medical | FP32    | Sep 25, | unlocke |
| rain_co | med/

In [7]:
# you can then filter on the model you want so it is easier to grab the repository path 
!ngc registry model list nvidia/med/*spleen_ct_segment*


+---------+---------+---------+---------+---------+---------+---------+---------+
| Name    | Reposit | Latest  | Applica | Framewo | Precisi | Last Mo | Permiss |
|         | ory     | Version | tion    | rk      | on      | dified  | ion     |
+---------+---------+---------+---------+---------+---------+---------+---------+
| clara_p | nvidia/ | 1       | Segment | Clara   | AMP     | May 20, | unlocke |
| t_splee | med/cla |         | ation   | Train   |         | 2021    | d       |
| n_ct_se | ra_pt_s |         |         |         |         |         |         |
| gmentat | pleen_c |         |         |         |         |         |         |
| ion     | t_segme |         |         |         |         |         |         |
|         | ntation |         |         |         |         |         |         |
+---------+---------+---------+---------+---------+---------+---------+---------+


The cell below will list the deepgrow models

In [11]:
# you can then filter on the model you want so it is easier to grab the repository path 
!ngc registry model list nvidia/med/*clara_pt_deepgrow*

+---------+---------+---------+---------+---------+---------+---------+---------+
| Name    | Reposit | Latest  | Applica | Framewo | Precisi | Last Mo | Permiss |
|         | ory     | Version | tion    | rk      | on      | dified  | ion     |
+---------+---------+---------+---------+---------+---------+---------+---------+
| clara_p | nvidia/ | 1       | Annotat | Clara   | AMP     | May 21, | unlocke |
| t_deepg | med/cla |         | ion     | Train   |         | 2021    | d       |
| row_3d_ | ra_pt_d |         |         |         |         |         |         |
| annotat | eepgrow |         |         |         |         |         |         |
| ion     | _3d_ann |         |         |         |         |         |         |
|         | otation |         |         |         |         |         |         |
| clara_p | nvidia/ | 1       | Annotat | Clara   | AMP     | May 21, | unlocke |
| t_deepg | med/cla |         | ion     | Train   |         | 2021    | d       |
| row_2d_ | ra_p

The cell below will download the the deep grow model from NGC and load it to AIAA

In [18]:
!AIAA load deepgrow3d ngc "nvidia/med/clara_pt_deepgrow_3d_annotation" 1


Loading Model: deepgrow3d
Loading deepgrow3d from NGC: https://ngc.nvidia.com/catalog/models/nvidia:med:clara_pt_deepgrow_3d_annotation
{
  "name": "deepgrow3d",
  "labels": [],
  "description": "3D DeepGrow model based on Unet",
  "version": "1",
  "type": "deepgrow",
  "deepgrow": "3D"
}


The following cell will download the spleen model and load it into the AIAA server

In [19]:
!AIAA load spleen_seg ngc "nvidia/med/clara_pt_spleen_ct_segmentation" 1


Loading Model: spleen_seg
Loading spleen_seg from NGC: https://ngc.nvidia.com/catalog/models/nvidia:med:clara_pt_spleen_ct_segmentation
{
  "name": "spleen_seg",
  "labels": [
    "spleen"
  ],
  "description": "A pre-trained model for volumetric (3D) segmentation of the spleen from CT image",
  "version": "1",
  "type": "segmentation"
}


In [21]:
# load lung models for annotation e segmentation (AMP precision ?)
!AIAA load lung_ann ngc "nvidia/med/clara_pt_covid19_ct_lung_annotation" 1
!AIAA load lung_seg ngc "nvidia/med/clara_pt_covid19_ct_lung_segmentation" 1


Loading Model: lung_ann
Loading lung_ann from NGC: https://ngc.nvidia.com/catalog/models/nvidia:med:clara_pt_covid19_ct_lung_annotation
{
  "name": "lung_ann",
  "labels": [
    "lung"
  ],
  "description": "A pre-trained model for volumetric (3D) segmentation of the lung from CT image using DEXtr3D",
  "version": "1",
  "type": "annotation"
}
Loading Model: lung_seg
Loading lung_seg from NGC: https://ngc.nvidia.com/catalog/models/nvidia:med:clara_pt_covid19_ct_lung_segmentation
{
  "name": "lung_seg",
  "labels": [
    "lung"
  ],
  "description": "A pre-trained model for volumetric (3D) segmentation of the Lung from CT image",
  "version": "1",
  "type": "segmentation"
}


## 3.3 Manually Download model from NGC 
If you have a private registry or have early access (EA) to nvidia's developer program, 
you can manually downloaded the MMAR form NGC by clicking the ... button as shown below
<br><img src="screenShots/DownloadFromNGC.png" alt="Drawing" style="height: 400px;"/><br>


**You should download zip file to /claraDevDay/AIAA/DownloadsFromNGC/**

For example if you downloaded the covid segmentation model and named the file `CovidsegModel.zip`, 
then you can run cell below to load up the zipped file to AIAA server 

**Please make sure the zip file directly contains the mmar folder structure 
Error will occur if the zipped file contains a folder before the mmar**

In [ ]:

downloadedFile="CovidsegModel.zip"

downloadDir=AIAA_ROOT+"../DownloadsFromNGC/"
modelName=downloadedFile[:-4]
zipmodelPath=downloadDir+downloadedFile
!AIAA load $modelName zip $zipmodelPath


## 3.3.1 (optional) In case of error with zip file
In case the zip files container an extra directory, you can unzip using cell below
Zip file should have the MMAR structure. 
Cell below would unzip the MMAR then upload it to AIAA server 
```
downloadedFile="deepgrow_3d.zip"

downloadDir=AIAA_ROOT+"../DownloadsFromNGC/"
modelName=downloadedFile[:-4]
modelFolderPath=downloadDir+modelName

%cd $downloadDir
!unzip $downloadedFile -d $modelName
# then you should load the mmar to AIAA 

!AIAA load $modelName mmar $modelFolderPath
```

## 3.4 load a pipeline created from multiple models into AIAA
AIAA now supports the concept of a pipelines which triggers multiple models one at a time.
cell below creates a pipeline out of 3D deepgrow which gets new seed points for multiple runs of deepgrow 2D.

In [18]:
modelName="GA_Deepgrow_Pipeline"
configFolderPath=AIAA_ROOT+"../Deepgrow2D3D_pipeline.json"
 
!AIAA load $modelName pipeline $configFolderPath 


Loading Model: GA_Deepgrow_Pipeline
Loading GA_Deepgrow_Pipeline from AIAA Config: /claraDevDay/AIAA/workspace/../Deepgrow2D3D_pipeline.json
{
  "error": {
    "message": [
      "CLASS_INIT_ERROR",
      "Instantiation of class LargestCCd failed: module 'aiaa.transforms' has no attribute 'LargestCCd'."
    ],
    "type": "AIAAException"
  },
  "success": false
}



## 3.5 Check for models loaded into AIAA
Let us check the server and check that the model is uploaded. 
Cell below would list models loaded by the AIAA server 

In [20]:
!AIAA list |grep name  

    "name": "Test",
    "name": "deepgrow3d",
    "name": "spleen_seg",


## 3.6 (Optional for advanced users) Loading models using curl commands
For advanced users or programmers integrating with AIAA, 
They could do the above task using curl commands as:  

```
http_str="http://127.0.0.1:"+AIAA_PORT+"/admin/model/"+modelName
conf_str="config=@"+downloadDir+modelName+"/config/config_aiaa.json;type=application/json"
dataArg="data=@"+downloadDir+modelName+"/models/model.ts"
cmd='curl -X PUT "'+http_str +'" -F "'+conf_str +'" -F "'+dataArg +'"'
```
Command below would download from NGC and load the deep grow and spleen segmentation model to AIAA
```
# model name from NGC is clara_train_deepgrow_aiaa_inference_only
http_str="http://127.0.0.1:"+AIAA_PORT+"/admin/model/clara_deepgrow"
!curl -X PUT $http_str \
     -H "accept: application/json" \
     -H "Content-Type: application/json" \
     -d '{"path":"nvidia/med/clara_train_deepgrow_aiaa_inference_only","version":"1"}'
```
```
http_str="http://127.0.0.1:"+AIAA_PORT+"/admin/model/clara_ct_seg_spleen"
!curl -X PUT $http_str \
     -H "accept: application/json" \
     -H "Content-Type: application/json" \
     -d '{"path":"nvidia/med/clara_ct_seg_spleen","version":"1"}'
```


# 4. AIAA Clients 

AIAA server can connect to any client that implements the APIs found [here](https://github.com/NVIDIA/ai-assisted-annotation-client
). 
NVIDIA has already implemented these APIs for a number of open source viewers as:


## 4.1. [3D Slicer](https://www.slicer.org/)

In order to use slicer you should: 
1. Install and setup slicer 3d following steps [here](https://github.com/NVIDIA/ai-assisted-annotation-client/tree/master/slicer-plugin)
    1. Download and install recent 3D Slicer Preview Release (4.11.x) from [here](http://download.slicer.org/).
    For Early access please use slicer 4.13 (unstable release) this is needed to enable 3d geepgrow models. 
    2. Start 3D Slicer and open the Extension manager
    3. Install NvidiaAIAssistedAnnotation extension (in Segmentation category), wait for the installation to complete, and click Restart
2. Install by searching for nvidia in the plugin manager
3. Configure plugin. Add the AIAA server location and make sure the session is enabled
   <br><img src="screenShots/SlicerConfig.png" alt="Drawing" style="height: 400px;"/>
4. You should load a volume and start trying the spleen and deep grow model as shown below
     
<br><img src="screenShots/Slicer.png" alt="Drawing" style="height: 400px;"/><br>

## 4.2. [MITK](https://www.mitk.org/wiki/The_Medical_Imaging_Interaction_Toolkit_(MITK))
MITK is another viewer that you can use with AIAA. 
You can download and install it [here](http://mitk.org/wiki/Downloads). 
Please make sure you install the release with nvidia AIAA.<br>
_Note_: Deep grow is not enabled yet in MITK 


## 4.3. [OHIF](https://docs.ohif.org/)
We have integrated AIAA with OHIF as a plugin, This has also been adapted in to XNAT. 
Plugin code can be found as a [branch of OHIF github](https://github.com/SachidanandAlle/Viewers). 

For more on how to use OHIF please see [OHIF Notebook](AIAAwOHIF.ipynb)

# 5. Delete Models from AIAA server  

In [ ]:
# if you need to change the model name 
#modelName="Deepgrow3DV4EA"
!AIAA delete $modelName


Or use the http API
```
http_str='"http://127.0.0.1:'+AIAA_PORT+'/admin/model/'+modelName+'"'
!curl -X DELETE $http_str
```


# 6. Stop AIAA server
In order to stop the AIAA server you can run cell below.

In [ ]:
!AIAA stop


However, this will **not free up your gpu memory** 
since we are using **triton which is running in another container.**
In order to release the gpu memory you would need to either stop the triton container, 
or delete the models from the triton model directory `/claraDevDay/AIAA/workspace/triton_models/`

In [ ]:
#!mkdir $AIAA_ROOT/triton_models_full
#!mv $AIAA_ROOT/triton_models_full_back/* $AIAA_ROOT/triton_models_full/


# Next steps
You should now train deepgrow yourself using 
[Train Deepgrow Notebook (2D and 3D)](AIAA/DeepGrow.ipynb) <span style="color:red">(New in V4)</span>


